In [60]:
import numpy as np
import pathlib
from nilearn.decomposition import CanICA
def ICA_decomposition(func_filenames, group, i):
    fast_ica = CanICA(n_components=30,
                    memory="nilearn_cache", memory_level=2,
                    mask_strategy='whole-brain-template',
                    do_cca=False,
                    random_state=0)
    fast_ica.fit(func_filenames)
    P = pathlib.Path('/home/niusha/Documents/Codes/local-experiment/io/ICAs')
    ICA_s = fast_ica.components_img_
    ICA_s.to_filename(P / (f'ICAs_{group}_{i}.nii.gz'))
    return ICA_s

In [89]:
from nilearn import masking
from nilearn.image import iter_img
from nilearn.masking import apply_mask

def Means_after_masking(ICAs,DBM_maps):

    means_after_mask = np.zeros((30,DBM_maps.shape[3]))
    for i, cur_img in enumerate(iter_img(ICAs)):
        mask_img = masking.compute_epi_mask(cur_img)
        means_after_mask[i,:] = np.mean(apply_mask(DBM_maps,mask_img),axis=1)
        
    return means_after_mask

In [62]:
import pandas as pd

subject_df = pd.read_csv("/home/niusha/Documents/Codes/subject_IDs.csv")

N = len(subject_df.ID)
ID_map = dict(zip(range(N),subject_df.ID))

Healthy_index = np.where(subject_df.PD == 0)
Healthy_subject = subject_df.ID.iloc[Healthy_index]

PD_index = np.where(subject_df.PD == 1)
PD_subject = subject_df.ID.iloc[PD_index]

In [63]:
from nilearn.image import load_img
Original_DBMs = load_img("/home/niusha/Documents/Codes/reshape_merged_dbm3_masked.nii")

In [64]:
from random import sample
from nilearn.image import index_img

PD_sample = sample((PD_index[0]).tolist(), len(PD_index[0]))
Healthy_sample = sample((Healthy_index[0]).tolist(), len(Healthy_index[0]))

PD_bootstrapped_cohort = index_img(Original_DBMs, PD_sample)
Healthy_bootstrapped_cohort = index_img(Original_DBMs, Healthy_sample)

PD_ICAs = ICA_decomposition(PD_bootstrapped_cohort, "PD", 1)
Healthy_ICAs = ICA_decomposition(Healthy_bootstrapped_cohort, "Healthy", 1)

/home/niusha/anaconda3/envs/py10/lib/python3.10/site-packages/joblib/memory.py:632: UserWarning: Cannot inspect object functools.partial(<function compute_multi_brain_mask at 0x7f4f99aca9e0>, mask_type='whole-brain'), ignore list will not work.
  return hashing.hash(filter_args(self.func, self.ignore, args, kwargs),
/home/niusha/anaconda3/envs/py10/lib/python3.10/site-packages/nilearn/masking.py:568: UserWarning: Persisting input arguments took 3.09s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 function (e.g. large strings).
THIS IS A JOBLIB ISSUE. If you can, kindly provide the joblib's team with an
 example so that they can fix the problem.
  resampled_template = cache(resampling.resample_to_img, memory)(
/home/niusha/anaconda3/envs/py10/lib/python3.10/site-packages/joblib/memory.py:810: UserWarning: Cannot inspect object functools.parti

In [90]:
PD_means = Means_after_masking(PD_ICAs,PD_bootstrapped_cohort)
Healthy_means = Means_after_masking(Healthy_ICAs,Healthy_bootstrapped_cohort)

(101, 119, 97, 30)
(101, 119, 97, 30)


In [91]:
from scipy.stats import ttest_ind

P_vals = np.zeros([30])
T_vals = np.zeros([30])
for i in range(30):
    T_vals[i], P_vals[i] = ttest_ind(PD_means[:,i], Healthy_means[:,i])


In [93]:
P_vals * 30

array([1.32736022e-25, 1.31426185e-38, 1.79223254e-48, 9.04914945e-24,
       6.83140773e-57, 3.94030939e-04, 4.00576297e-18, 1.17997173e-45,
       2.33091257e-54, 7.88759613e-17, 7.12293729e-37, 6.66396254e-42,
       1.91530642e-37, 1.62190707e+01, 2.31767046e-50, 1.45812100e-23,
       2.15466993e-09, 1.81134446e-41, 1.65636613e-06, 2.04879978e-10,
       1.63058394e-45, 4.30232938e-43, 8.51303443e-55, 1.79313472e-35,
       4.26283853e-63, 1.26499302e-08, 1.67470315e-29, 4.66986862e-36,
       1.87524975e-47, 3.14024712e-45])

In [ ]:
# from nilearn.plotting import plot_prob_atlas

# # Plot all ICA components together
# plot_prob_atlas(canica_components_img, title='All ICA components')

# from nilearn.image import iter_img
# from nilearn.plotting import plot_stat_map, show

# for i, cur_img in enumerate(iter_img(canica_components_img)):
#     plot_stat_map(cur_img, display_mode="z", title="IC %d" % i,
#                   cut_coords=1, colorbar=False)

In [ ]:
# from nilearn.glm import threshold_stats_img

# ICA_tr = threshold_stats_img(stat_img=ICAs, threshold=3.0)
#The output is not compatible with mask and also I dont know what is the ouput of ICA. I tought it would be z stat, but when I set thr=3 I receive warning regarding the amount